In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setting project path
import os
import sys

gparent = os.path.join(os.pardir, os.pardir)
sys.path.append(gparent)

# imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_sm_pipeline

import matplotlib.pyplot as plt
import seaborn as sns

# setting style
sns.set_theme('talk')
plt.style.use('fivethirtyeight')
sns.set_palette(palette='Blues_r')

In [3]:
path = os.path.join(gparent, 'data/processed', 'cleaned.csv')
test_df = pd.read_csv(path, keep_default_na=False)

In [4]:
test_df.head(2)

,Target,Subject Age Group,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,1,NA,Arrest,None,7500,1984,M,Black,Asian,Male,NA,NA,NA,SOUTH PCT 1ST W - ROBERT,0,0,South,O,O2
1,0,NA,Field Contact,None,5670,1965,M,White,NA,NA,NA,NA,NA,NA,0,0,NA,NA,NA


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46960 entries, 0 to 46959
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Target                    46960 non-null  int64 
 1   Subject Age Group         46960 non-null  object
 2   Stop Resolution           46960 non-null  object
 3   Weapon Type               46960 non-null  object
 4   Officer ID                46960 non-null  object
 5   Officer YOB               46960 non-null  int64 
 6   Officer Gender            46960 non-null  object
 7   Officer Race              46960 non-null  object
 8   Subject Perceived Race    46960 non-null  object
 9   Subject Perceived Gender  46960 non-null  object
 10  Initial Call Type         46960 non-null  object
 11  Final Call Type           46960 non-null  object
 12  Call Type                 46960 non-null  object
 13  Officer Squad             46960 non-null  object
 14  Arrest Flag           

In [6]:
column_list = test_df.columns
excluded = ['Stop Resolution', 'Weapon Type', 'Officer ID',
          'Initial Call Type', 'Final Call Type', 'Officer Squad',
            'Precinct', 'Sector', 'Call Type', 'Arrest Flag', 
            'Frisk Flag', 'Beat']
cols = [x for x in column_list if x not in excluded]

In [7]:
cols

['Target',
 'Subject Age Group',
 'Officer YOB',
 'Officer Gender',
 'Officer Race',
 'Subject Perceived Race',
 'Subject Perceived Gender']

In [8]:
test_df = test_df[cols]

In [9]:
test_df.head()

,Target,Subject Age Group,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender
0,1,NA,1984,M,Black,Asian,Male
1,0,NA,1965,M,White,NA,NA
2,0,NA,1961,M,White,White,Male
3,0,NA,1963,M,White,NA,NA
4,0,NA,1977,M,White,Black,Male


In [10]:
X = test_df.drop('Target', axis=1)
y = test_df['Target']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=2021,
                                                    stratify=y
                                                   )

In [12]:
string_selector = make_column_selector(dtype_include='object')
number_selector = make_column_selector(dtype_include='number', dtype_exclude='object')

In [13]:
preprocessing = make_column_transformer((OneHotEncoder
                                         (handle_unknown='ignore'),string_selector),
                                          (MinMaxScaler(), number_selector))

In [14]:
preprocessing.fit_transform(X_train)

<35220x36 sparse matrix of type '<class 'numpy.float64'>'
	with 211291 stored elements in Compressed Sparse Row format>

In [15]:
preprocessing.transformers_[0][1].get_feature_names()

array(['x0_1 - 17', 'x0_18 - 25', 'x0_26 - 35', 'x0_36 - 45',
       'x0_46 - 55', 'x0_56 and Above', 'x0_NA', 'x1_F', 'x1_M', 'x1_N',
       'x2_Asian', 'x2_Black', 'x2_Hispanic', 'x2_Multi-Racial', 'x2_NA',
       'x2_N_American', 'x2_P_Islander', 'x2_Unknown', 'x2_White',
       'x3_Asian', 'x3_Black', 'x3_Hispanic', 'x3_Multi-Racial', 'x3_NA',
       'x3_N_American', 'x3_Other', 'x3_P_Islander', 'x3_Unknown',
       'x3_White', 'x4_Female', 'x4_Gender Diverse', 'x4_Male', 'x4_NA',
       'x4_Undetermined', 'x4_Unknown'], dtype=object)

In [16]:
clf = LogisticRegression()
pipeline = make_pipeline(preprocessing, clf)

In [17]:
cross_val_score( pipeline, X_train, y_train, scoring='precision')

/Users/dsbox/opt/anaconda3/envs/terry_stops/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/dsbox/opt/anaconda3/envs/terry_stops/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dsbox/opt/anaconda3/envs/terry_stops/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbf

array([0., 0., 0., 0., 0.])

In [18]:
sm = SMOTE(random_state=2021)

In [19]:
pipeline2 = make_sm_pipeline(preprocessing, sm, clf)

In [20]:
cross_val_score(pipeline2, X_train, y_train, scoring='precision')

/Users/dsbox/opt/anaconda3/envs/terry_stops/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/dsbox/opt/anaconda3/envs/terry_stops/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

array([0.29624872, 0.28493365, 0.29127726, 0.28791037, 0.29413305])

In [21]:
clf2 = LogisticRegression(max_iter=275)

In [22]:
pipeline3 = make_sm_pipeline(preprocessing, sm, clf2)

In [23]:
cross_val_score(pipeline3, X_train, y_train, scoring='precision')

array([0.29632485, 0.28426528, 0.29127726, 0.28764982, 0.29416383])

In [24]:
preprocessing2 = make_column_transformer((OneHotEncoder
                                         (handle_unknown='ignore'),string_selector),
                                          (StandardScaler(), number_selector))

In [25]:
pipeline4 = make_sm_pipeline(preprocessing2, sm, clf2)

In [26]:
cross_val_score(pipeline4, X_train, y_train, scoring='precision')

array([0.29623905, 0.28489434, 0.29041451, 0.28754244, 0.29485505])

In [27]:
def pre_score(y_true, y_pred):
    
    precision = precision_score(y_true, y_pred)

    return precision

precision = make_scorer(pre_score)

class HarnessCCV:
    
    def __init__(self, scorer, random_state=2021):
        self.scorer = scorer
        self.history = pd.DataFrame(columns=['Name', 'Accuracy', 'Notes'])

    def report(self, estimator, X, y, name, notes=''):
        # Create a list to hold the scores from each fold
        kfold_val_scores = np.ndarray(5)
        kfold_train_scores = np.ndarray(5)

        # Instantiate a splitter object and loop over its result
        kfold = StratifiedKFold(n_splits=5)
        for fold, (train_index, val_index) in enumerate(kfold.split(X, y)):
            # Extract train and validation subsets using the provided indices
            X_t, X_val = X.iloc[train_index], X.iloc[val_index]
            y_t, y_val = y.iloc[train_index], y.iloc[val_index]
        
            # Instantiate StandardScaler
            scaler = StandardScaler()
            # Fit and transform X_t
            X_t_scaled = scaler.fit_transform(X_t)
            # Transform X_val
            X_val_scaled = scaler.transform(X_val)
        
            # Instantiate SMOTE
            sm = SMOTE(random_state=2021)
            # Fit and transform X_t_scaled and y_t using sm
            X_t_oversampled, y_t_oversampled = sm.fit_resample(X_t_scaled, y_t)
        
            # Clone the provided model and fit it on the train subset
            temp_model = clone(estimator)
            temp_model.fit(X_t_oversampled, y_t_oversampled)
        
            # Evaluate the model on the validation subsets
            score_train = precision_score(temp_model.predict(X_t_oversampled), y_t_oversampled)
            scores_val = precision_score(temp_model.predict(X_val_scaled), y_val)
            kfold_train_scores[fold] = score_train
            kfold_val_scores[fold] = scores_val
        
        frame = pd.DataFrame([[name, scores_val.mean(), notes]], columns=['Name', 'Accuracy', 'Notes'])
        self.history = self.history.append(frame)
        self.history = self.history.reset_index(drop=True)
        self.history = self.history.sort_values('Accuracy')
        self.print_error(name, scores_val.mean())
        print(kfold_val_scores)
                               
        return kfold_train_scores, kfold_val_scores 

    def print_error(self, name, Accuracy):
        print(f'{name} has an average percision of {Accuracy}')

In [28]:
modeling = HarnessCCV(precision)

In [29]:
clf3 = KNeighborsClassifier()

In [30]:
pipeline5 = make_sm_pipeline(preprocessing2, sm, clf3)

In [31]:
FE = X_train.copy()

In [32]:
FE = FE.reset_index(drop=True)
FE = pd.get_dummies(FE)

In [33]:
modeling.report(clf3, FE, y_train, 'KNN')

KNN has an average percision of 0.48105625717566014
[0.4606548  0.46927053 0.42365098 0.47359357 0.48105626]


(array([0.69494083, 0.72087321, 0.64780764, 0.68882603, 0.66850542]),
 array([0.4606548 , 0.46927053, 0.42365098, 0.47359357, 0.48105626]))

In [34]:
modeling.history

,Name,Accuracy,Notes
0,KNN,0.481056,
